The user sends a query about Arize to your service.

langchain.embeddings.OpenAIEmbeddings makes a request to OpenAI to embed the user query using the text-embedding-ada-002 model.

We retrieve by searching against the entries of your Qdrant database for the most similar pieces of context by MMR.

langchain.llms.ChatOpenAI generates a response by formatting the query and retrieved context into a single prompt and sending a request to OpenAI with the gpt-4-turbo-preview model.

The response is returned to the user.

In [ ]:
!pip install langchain qdrant-client langchain_community tiktoken cohere langchain-openai "protobuf>=3.20.3"

In [3]:
# Third-party library imports
import nest_asyncio
import numpy as np
import pandas as pd

# Phoenix imports
import phoenix as px
from langchain.callbacks import StdOutCallbackHandler

# LangChain imports
from langchain.chains import RetrievalQA
from langchain.document_loaders import GitbookLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_openai import ChatOpenAI
from phoenix.trace.langchain import LangChainInstrumentor

# Miscellaneous imports

# Configuration and Initialization
nest_asyncio.apply()
pd.set_option("display.max_colwidth", None)

/media/uberdev/ddrv/telemetenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from dotenv import load_dotenv
load_dotenv("/media/uberdev/ddrv/gitFolders/python_de_learners_data/.env")

True

In [6]:
import os
model_embed = "text-embedding-ada-002"
embeddings = OpenAIEmbeddings(model=model_embed, 
                              api_key=os.environ["OPENAI_API_KEY"])

/media/uberdev/ddrv/telemetenv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
def load_gitbook_docs(docs_url):
    """
    Loads documentation from a Gitbook URL.
    """

    loader = GitbookLoader(
        docs_url,
        load_all_paths=True,
    )
    return loader.load()


docs_url = "https://docs.arize.com/arize/"
docs = load_gitbook_docs(docs_url)

/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)
Fetching pages: 100%|##########| 260/260 [02:59<00:00,  1.45it/s]


In [12]:
# len(docs)
len(docs[0].page_content.split(" "))

109

In [13]:
# embedding is going to happen in this area...
# take care of the documents you are sending 
# through the OpenAI embedding model
qdrant = Qdrant.from_documents(
    docs[:10], # only embed 10 documents
    embeddings,
    location=":memory:",
    collection_name="arize_docs",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [15]:
LangChainInstrumentor().instrument()

In [16]:
# building the langchain appln

handler = StdOutCallbackHandler()

num_retrieved_documents = 2

retriever = qdrant.as_retriever(
    search_type="mmr", search_kwargs={"k": num_retrieved_documents}, enable_limit=True
)

chain_type = "stuff"  # stuff, refine, map_reduce, and map_rerank

chat_model_name = "gpt-4o-mini"

In [17]:
llm = ChatOpenAI(model_name=chat_model_name,
                 temperature=0.0)

In [18]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type=chain_type,
    retriever=retriever,
    metadata={"application_type": "question_answering"},
    callbacks=[handler],
)

In [19]:
query_df = pd.read_parquet(
    "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/langchain/langchain_query_dataframe_with_user_feedbackv2.parquet"
)

In [20]:
(session := px.launch_app()).view()

INFO:phoenix.config:📋 Ensuring phoenix working directory: /home/uberdev/.phoenix


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
📺 Opening a view to the Phoenix app. The app is running at http://localhost:6006/


In [21]:
for i in range(2):
    row = query_df.iloc[i]
    response = chain.invoke(row["text"])
    print(response)



> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'How do I use the SDK to upload a ranking model?', 'result': "I don't know."}


> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'What drift metrics are supported in Arize?', 'result': "I don't know."}


In [22]:
from phoenix.evals import (
    RAG_RELEVANCY_PROMPT_RAILS_MAP,
    RAG_RELEVANCY_PROMPT_TEMPLATE,
    OpenAIModel,
    llm_classify,
)

In [23]:
# create evaluation dataframes with "input" and "reference" columns
context0_eval_df = query_df.copy()
context0_eval_df["input"] = context0_eval_df["text"]
context0_eval_df["reference"] = context0_eval_df["context_text_0"]

context1_eval_df = query_df.copy()
context1_eval_df["input"] = context1_eval_df["text"]
context1_eval_df["reference"] = context1_eval_df["context_text_1"]

In [24]:
RAG_RELEVANCY_PROMPT_RAILS_MAP

OrderedDict([(True, 'relevant'), (False, 'unrelated')])

In [25]:
RAG_RELEVANCY_PROMPT_TEMPLATE


You are comparing a reference text to a question and trying to determine if the reference text
contains information relevant to answering the question. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {input}
    ************
    [Reference text]: {reference}
    ************
    [END DATA]
Compare the Question above to the Reference text. You must determine whether the Reference text
contains information that can answer the Question. Please focus on whether the very specific
question can be answered by the information in the Reference text.
Your response must be single word, either "relevant" or "unrelated",
and should not contain any text or characters aside from that word.
"unrelated" means that the reference text does not contain an answer to the Question.
"relevant" means the reference text contains an answer to the Question.

In [ ]:
model = OpenAIModel(model="gpt-4o-mini")
#  A novel idea is to use LLMs to evaluate retrieval quality 
# by simply asking the LLM whether each piece of retrieved
# context is relevant or irrelevant to the corresponding 
# query.

context0_relevance = llm_classify(
    context0_eval_df,
    template=RAG_RELEVANCY_PROMPT_TEMPLATE,
    rails=list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values()),
    provide_explanation=True,
    model=model,
)
context1_relevance = llm_classify(
    context1_eval_df,
    template=RAG_RELEVANCY_PROMPT_TEMPLATE,
    rails=list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values()),
    provide_explanation=True,
    model=model,
)

In [ ]:
sample_query_df = query_df.copy()
sample_query_df["openai_relevance_0"] = context0_relevance["label"]
sample_query_df["openai_relevance_1"] = context1_relevance["label"]

In [ ]:
num_relevant_documents_array = np.zeros(len(sample_query_df))
num_retrieved_documents = 2

In [ ]:
for retrieved_document_index in range(0, num_retrieved_documents):
    num_retrieved_documents = retrieved_document_index + 1
    num_relevant_documents_array += (
        sample_query_df[f"openai_relevance_{retrieved_document_index}"]
        .map(lambda x: int(x == "relevant"))
        .to_numpy()
    )
    sample_query_df[f"openai_precision@{num_retrieved_documents}"] = pd.Series(
        num_relevant_documents_array / num_retrieved_documents
    )

In [ ]:
sample_query_df[
    [
        "openai_relevance_0",
        "openai_relevance_1",
        "openai_precision@1",
        "openai_precision@2",
    ]
]